In [ ]:
Schema Creation

In [ ]:
import pymysql
import pandas as pd

def create_tables(cursor):

    cursor.execute("DROP DATABASE IF EXISTS walmart_test")
    cursor.execute("CREATE DATABASE walmart_test")
    cursor.execute("USE walmart_test")
    
    # Create tables
    cursor.execute("""
        CREATE TABLE DimCustomer (
            Customer_ID INT PRIMARY KEY,
            Gender VARCHAR(10),
            Age VARCHAR(10),           -- "0-17", "55+", "26-35" etc.
            Occupation VARCHAR(50),    -- "10", "16", "15" etc. (treated as categories)
            City_Category VARCHAR(10), -- "A", "B", "C"
            Stay_In_Current_City_Years VARCHAR(10), -- "2", "4", "3", "0" etc.
            Marital_Status INT 
        )
    """)
    
    cursor.execute("""
        CREATE TABLE DimProduct (
            Product_ID VARCHAR(10) PRIMARY KEY,
            Product_Category VARCHAR(50),
            Price float,
            StoreID INT,
            SupplierID INT,
            StoreName VARCHAR(100),
            SupplierName VARCHAR(100)
        )
    """)
    
    cursor.execute("""
        CREATE TABLE DimDate (
            Date_ID DATE PRIMARY KEY,
            Year INT,
            Month INT,
            Day INT,
            Weekday_Name VARCHAR(20),
            Is_Weekend TINYINT(1),
            Quarter INT
        )
    """)
    
    cursor.execute("""
        CREATE TABLE FactSales (
            Sales_ID INT AUTO_INCREMENT PRIMARY KEY,
            -- Core transactional data
            Customer_ID INT,
            Product_ID VARCHAR(10),
            Order_ID INT,
            Date_ID DATE,
            Quantity INT,
            -- Enriched customer demographics (from DimCustomer)
            Gender VARCHAR(10),
            Age VARCHAR(10),
            Occupation VARCHAR(50),
            City_Category VARCHAR(10),
            Stay_In_Current_City_Years VARCHAR(10),
            Marital_Status INT,
            -- Enriched product details (from DimProduct)
            Product_Category VARCHAR(50),
            Price DECIMAL(10,2),
            StoreID INT,
            StoreName VARCHAR(100),
            SupplierID INT,
            SupplierName VARCHAR(100),
            -- Calculated measures
            Total_Amount DECIMAL(10,2),
            -- Foreign Keys for integrity
            FOREIGN KEY (Customer_ID) REFERENCES DimCustomer(Customer_ID),
            FOREIGN KEY (Product_ID) REFERENCES DimProduct(Product_ID),
            FOREIGN KEY (Date_ID) REFERENCES DimDate(Date_ID)
        )
    """)

def load_data(cursor):
    
    
    # Load customer data
    print("Loading customer data...")
    customer_df = pd.read_csv('customer_master_data.csv', index_col=0)
    
    for _, row in customer_df.iterrows():
        cursor.execute("""
            INSERT INTO DimCustomer (Customer_ID, Gender, Age, Occupation, City_Category, Stay_In_Current_City_Years, Marital_Status)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (row['Customer_ID'], row['Gender'], row['Age'], row['Occupation'], 
              row['City_Category'], row['Stay_In_Current_City_Years'], row['Marital_Status']))
    
    # Load product data  
    print("Loading product data...")
    product_df = pd.read_csv('product_master_data.csv')
    
    for _, row in product_df.iterrows():
        cursor.execute("""
            INSERT INTO DimProduct (Product_ID, Product_Category, Price, StoreID, SupplierID, StoreName, SupplierName)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (row['Product_ID'], row['Product_Category'], row['price$'], row['storeID'],
              row['supplierID'], row['storeName'], row['supplierName']))
    
    print("Dimension tables loaded successfully!")

def load_date_dimension(cursor):
   
    print("Loading date dimension...")
    
    # Read transactional data to get all unique dates
    transactional_df = pd.read_csv('transactional_data.csv')
    
    # Extract unique dates
    unique_dates = pd.to_datetime(transactional_df['date']).unique()
    
    for date in unique_dates:
        date_str = date.strftime('%Y-%m-%d')
        year = date.year
        month = date.month
        day = date.day
        weekday_name = date.strftime('%A')
        is_weekend = 1 if date.weekday() >= 5 else 0  # 5=Saturday, 6=Sunday
        quarter = (date.month - 1) // 3 + 1
        
        cursor.execute("""
            INSERT INTO DimDate (Date_ID, Year, Month, Day, Weekday_Name, Is_Weekend, Quarter)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (date_str, year, month, day, weekday_name, is_weekend, quarter))
    
    print("Date dimension loaded successfully!")

def main():
    conn = None
    
    try:
        host = input("Enter database host [localhost]: ") or "localhost"
        user = input("Enter database username [root]: ") or "root"
        password = input("Enter database password: ")
        
      
        
        conn = pymysql.connect(
            host=host,
            user=user,
            password=password,
            charset='utf8mb4'
        )
        cursor = conn.cursor()
        
        create_tables(cursor)
        load_data(cursor)
        load_date_dimension(cursor) 
        
        conn.commit()
        print("Database setup completed successfully!")
        
    except Exception as e:
        print(f"Error: {e}")
    finally:
        if conn:
            conn.close()

if __name__ == "__main__":
    main()

Enter database host [localhost]: localhost
Enter database username [root]: root
Enter database password: yumn@1922@mn@
Loading customer data...
Loading product data...
Dimension tables loaded successfully!
Loading date dimension...
Date dimension loaded successfully!
Database setup completed successfully!
